In [137]:
import json
import pandas as pd
from numpy import array

import torch
import torch.nn
import torch.optim as optim

from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [138]:
file_path = "C:\\Users\\dcrai\\source\\repos\\DATA698\\Code\\Data\\data.json"
file_path_hero = "C:\\Users\\dcrai\\source\\repos\\DATA698\\Code\\Data\\hero_id_table.csv"
#file_csv = "C:\\Users\\dcrai\\source\\repos\\DATA698\\Code\\Data\\iter_1.csv"
#data = json.loads(file)

with open(file_path, 'r') as file:
    data = json.load(file)


heroes= pd.read_csv(file_path_hero)
# Now 'data' contains the contents of the JSON file


In [139]:
# Extract 'match_id', 'hero_id', and 'gold_t' from each element in 'data'
match_ids = [element['match_id'] for element in data]
hero_ids = [element['hero_id'] for element in data]
gold_t_values = [element['gold_t'] for element in data]

# Create a DataFrame from the extracted values
df = pd.DataFrame({'match_id': match_ids, 'hero_id': hero_ids, 'gold_t': gold_t_values})

# Displ

#7517376613

In [140]:
df

,match_id,hero_id,gold_t
0,7517376613,26,"[0, 170, 260, 350, 440, 827, 1051, 1141, 1278,..."
1,7517376613,84,"[0, 247, 580, 826, 1235, 1481, 1876, 2279, 265..."
2,7517376613,25,"[0, 357, 715, 1130, 1578, 1968, 2261, 2861, 33..."
3,7517376613,8,"[0, 246, 591, 895, 1255, 1499, 1823, 2117, 241..."
4,7517376613,5,"[0, 170, 281, 395, 559, 699, 821, 937, 1148, 1..."
...,...,...,...
10165,7634079912,10,"[0, 245, 553, 935, 1262, 1593, 2021, 2489, 295..."
10166,7634079912,55,"[0, 209, 445, 744, 1168, 1593, 1953, 2531, 275..."
10167,7634079912,98,"[0, 332, 654, 1046, 1536, 1828, 2285, 3071, 35..."
10168,7634079912,45,"[0, 209, 299, 469, 589, 789, 971, 1186, 1392, ..."


In [141]:
#df_match = df[df['match_id'] == 7517376613]

df_match = df[:300]

#df_match

df_subset = df_match[['hero_id', 'gold_t']].copy()

df_t = df_subset.T

#df_t.columns
# explode = df_t.explode(26)

# explode
df_subset

,hero_id,gold_t
0,26,"[0, 170, 260, 350, 440, 827, 1051, 1141, 1278,..."
1,84,"[0, 247, 580, 826, 1235, 1481, 1876, 2279, 265..."
2,25,"[0, 357, 715, 1130, 1578, 1968, 2261, 2861, 33..."
3,8,"[0, 246, 591, 895, 1255, 1499, 1823, 2117, 241..."
4,5,"[0, 170, 281, 395, 559, 699, 821, 937, 1148, 1..."
...,...,...
295,44,"[0, 205, 630, 921, 1201, 1414, 1577, 1913, 233..."
296,17,"[0, 327, 534, 965, 1215, 1398, 1738, 2000, 287..."
297,5,"[0, 170, 260, 574, 704, 840, 992, 1131, 1266, ..."
298,43,"[0, 208, 543, 866, 1161, 1439, 1775, 2138, 281..."


### TimeSeriesDataset Building

In [142]:
dataset.time_series

[tensor([   0,  170,  260,  350,  440,  827, 1051, 1141, 1278, 1368, 1593, 2076,
         2166, 2261, 2609, 2704, 2834, 3702, 4056, 4214, 4309, 4640, 4806, 5091,
         5451, 5635, 5790, 5943]),
 tensor([    0,   247,   580,   826,  1235,  1481,  1876,  2279,  2656,  2944,
          3407,  4263,  4601,  5348,  5885,  5980,  6623,  7232,  7862,  8065,
          8160,  8759,  9375,  9470, 10310, 10659, 10754, 11208]),
 tensor([    0,   357,   715,  1130,  1578,  1968,  2261,  2861,  3399,  4064,
          4468,  4937,  5415,  6018,  6399,  7117,  7670,  7939,  8517,  9208,
          9912, 10539, 11221, 11718, 12769, 13071, 13780, 14465]),
 tensor([    0,   246,   591,   895,  1255,  1499,  1823,  2117,  2411,  2876,
          3486,  3784,  4322,  4773,  5383,  6011,  6585,  7247,  7980,  8308,
          8403,  9349,  9872, 10530, 11037, 11754, 11849, 11944]),
 tensor([   0,  170,  281,  395,  559,  699,  821,  937, 1148, 1390, 1719, 1809,
         1899, 2053, 2237, 2378, 2528, 2758, 29

In [143]:
from torch.nn.utils.rnn import pack_padded_sequence
pad_test = [torch.tensor(ts) for ts in df_subset['gold_t']]

pad_test
#pack_padded_sequence(input, lengths, batch_first=False, enforce_sorted=True)


[tensor([   0,  170,  260,  350,  440,  827, 1051, 1141, 1278, 1368, 1593, 2076,
         2166, 2261, 2609, 2704, 2834, 3702, 4056, 4214, 4309, 4640, 4806, 5091,
         5451, 5635, 5790, 5943]),
 tensor([    0,   247,   580,   826,  1235,  1481,  1876,  2279,  2656,  2944,
          3407,  4263,  4601,  5348,  5885,  5980,  6623,  7232,  7862,  8065,
          8160,  8759,  9375,  9470, 10310, 10659, 10754, 11208]),
 tensor([    0,   357,   715,  1130,  1578,  1968,  2261,  2861,  3399,  4064,
          4468,  4937,  5415,  6018,  6399,  7117,  7670,  7939,  8517,  9208,
          9912, 10539, 11221, 11718, 12769, 13071, 13780, 14465]),
 tensor([    0,   246,   591,   895,  1255,  1499,  1823,  2117,  2411,  2876,
          3486,  3784,  4322,  4773,  5383,  6011,  6585,  7247,  7980,  8308,
          8403,  9349,  9872, 10530, 11037, 11754, 11849, 11944]),
 tensor([   0,  170,  281,  395,  559,  699,  821,  937, 1148, 1390, 1719, 1809,
         1899, 2053, 2237, 2378, 2528, 2758, 29

In [152]:
import torch
from torch.utils.data import Dataset

class TimeSeriesDataset(Dataset):
    # Class to create our dataset
    def __init__(self, df):
        self.hero_ids = df['hero_id'].values # Declaring hero_id values
        self.time_series = [torch.tensor(ts) for ts in df['gold_t']] # Converting the time_series into Tensors
        self.max_length = max(len(ts) for ts in self.time_series) # Grabs max length of all the tensors to pad them with 0s later

        # Check the dimensionality of the time series tensors
        for ts in self.time_series:
            print(f"Time series shape: {ts.shape}, Dimensions: {ts.dim()}")


    def __len__(self):
        return len(self.hero_ids) # Convenient length call
    

    def __getitem__(self, idx):
        hero_id = self.hero_ids[idx]
        time_series = self.time_series[idx]
        length = len(time_series)

        padded_time_series = torch.zeros(self.max_length)
        padded_time_series[:length] = time_series

        return hero_id, padded_time_series, length

"""    
def __getitem__(self, idx): # getitem is called by the PyTorch Dataloader
        # For the DataLoader, indexes are called to pull time-series data. Since we have the categorical variable in hero_id, we need to ensure
        #     to pull both the hero_id and the associated index of the time series. To do that the DataLoader passes the __getitem__ method an
        #     index, and __getitem__ returns the hero_id and the time_series at that index. In a time-series without categorical data, it'd normally
        #     just be the time series that is returned.
        hero_id = self.hero_ids[idx]
        time_series = self.time_series[idx]
        length = len(time_series)

        # Below is where padding is handled.
        #padded_time_series = torch.zeros(self.max_length, time_series.size(0)) # Creates a time-series of zeros of the longest series in the set
        padded_time_series = torch.zeros(self.max_length, time_series.size(1)) # Edited version?
        padded_time_series[:length, :] = time_series # EDited version?
        #padded_time_series[:length] = time_series #assigns the values from the individual time_series to the first 
        
        return hero_id, padded_time_series, length
 """

"    \ndef __getitem__(self, idx): # getitem is called by the PyTorch Dataloader\n        # For the DataLoader, indexes are called to pull time-series data. Since we have the categorical variable in hero_id, we need to ensure\n        #     to pull both the hero_id and the associated index of the time series. To do that the DataLoader passes the __getitem__ method an\n        #     index, and __getitem__ returns the hero_id and the time_series at that index. In a time-series without categorical data, it'd normally\n        #     just be the time series that is returned.\n        hero_id = self.hero_ids[idx]\n        time_series = self.time_series[idx]\n        length = len(time_series)\n\n        # Below is where padding is handled.\n        #padded_time_series = torch.zeros(self.max_length, time_series.size(0)) # Creates a time-series of zeros of the longest series in the set\n        padded_time_series = torch.zeros(self.max_length, time_series.size(1)) # Edited version?\n        pad

Playing with how the __getitem__ method works below

In [119]:
# ts_test = [torch.tensor(ts) for ts in df_subset['gold_t']] # Converting the time_series into Tensors
# length = len(ts_test[0])
# max_length = max(len(ts) for ts in ts_test)
# print(ts_test[0].shape)
# #print(ts_test[0].size(1))

# padded_test = torch.zeros(max_length, ts_test[0].size(0)) 

# padded_test[:length] = ts_test[0]

# padded_test
# #padded_test.size(1)
# # ts_test[1].size(1)
# # max_length = max(len(ts) for ts in ts_test)

# # print(max_length)
# # print(ts_test[1].size(1))

# # len(ts_test)

### Embedding Processing

Embedding will allow the model to develop a representation for each hero_id, with like hero_ids having similar embedding vectors. These vectors will help represent the differences between carries, mids, offlaners, and more.

Rule of Thumb: A common rule of thumb is to set the embedding_dim to be at least as large as the logarithm of the number of classes or groups you want to represent or expect. In your case, with around 5 groups, the logarithm of 5 is approximately 2. Since I know there are 5 roles, and I'd like to see them expressed atleast by role the log would be an embed vector of size 2.

Rule of Thumb (NLP): A common rule of thumb in natural language processing (NLP) tasks, where embeddings are widely used, is to set the embedding dimension to be approximately the fourth root of the vocabulary size (number of unique tokens/entities). While this is not a hard rule, it provides a reasonable ballpark estimate. In your case, with 124 unique processes, the fourth root of 124 is approximately 5. Multiplying this by a factor (e.g., 16 or 32, which are common embedding sizes) gives you a range of 80-160 as a reasonable starting point for the embedding dimension.

In many deep learning applications, embedding dimensions of 64, 128, or 256 are used.

I will try embed sizes of 20, 32, 64, 128, and 256.

In [159]:
import torch
import torch.nn as nn

class ProcessEmbedding(nn.Module):
    def __init__(self, df, embedding_dim):
        super(ProcessEmbedding, self).__init__() 
        """
        super() calls the intialization of the parent class of ProcessEmbedding, in this case nn.Module (PyTorch's class for all nnets)
        This is done to initizialize the class correctly. If not called, the nn.Module's functionalities will not work.

        """
        self.num_processes = len(df['hero_id'].unique()) # declaring number of different categories of time-series for dimensionialty reasons
        self.embedding_dim = embedding_dim # passing our embed size to be a class attribute
        self.process_embeddings = nn.Embedding(self.num_processes, embedding_dim)

        """ nn.Embedding creates and stores the embedding vector , it takes two arguments;
                    num_processes - the number of different embeddings it will need to hold (for us it would be the number of hero_ids = 124)
                    embedding_dim - the size of the vector of the embedding
        """


        self.hero_id_to_idx = {hero_id: idx for idx, hero_id in enumerate(df['hero_id'].unique())}

        """  self.hero_id_to_idx; converts hero_id to an integer index
        - is created since nn.Embedding expects an integer for an index value where an embedding is stored
        - this attribute effectively converts hero_ids to an index value that corresponds to the Embedding vectors row of values for that hero_id

        enumerate(df['hero_id'].unique()) iterates over the unique hero_id values and assigns a sequential index (idx) to each value, starting from 0. 
        It returns a list of tuples (idx, hero_id).
        For example, if the unique hero_id values are ['026', '184', '225', '38'], the enumerate function will return:
        [(0, '026'), (1, '184'), (2, '225'), (3, '38')]
        
        """
        

    def forward(self, hero_ids):
        """
        Args:
            hero_ids (Tensor or List): A tensor or list of hero IDs.
        Returns:
            Tensor: A tensor of shape (batch_size, embedding_dim) containing the process embeddings.

        We use the embedding with the forward method below.
            1. We convert the input hero_ids to their corresponding indices (process_ids) in the Embed Vector using the self.hero_id_to_idx mapping.
            2. We create a PyTorch tensor process_ids from the indices.
            3. We pass this process_ids tensor to the self.process_embeddings module, which retrieves and returns the corresponding embedding vectors.
        """
        process_ids = []
        print(hero_ids)
        for hero_id in hero_ids:
            process_id = self.hero_id_to_idx[hero_id.item()]
            if process_id not in process_ids: # this will ensure that only unique values are added to the list
                process_ids.append(process_id)
        
        # process_ids = [self.hero_id_to_idx[hero_id] for hero_id in hero_ids] # convert hero_ids
        process_ids = torch.tensor(process_ids) # Create tensor of process_ids
        return self.process_embeddings(process_ids) # Pass the process_ids to self.process_embeddings
    

    # def forward(self, hero_ids):
    #     process_ids = []
    #     for hero_id in hero_ids:
    #         try:
    #             process_id = self.hero_id_to_idx[hero_id.item()]
    #             process_ids.append(process_id)
    #         except KeyError:
    #             print(f"Error: hero_id {hero_id.item()} not found in hero_id_to_idx dictionary.")
    #             raise

    #     process_ids = torch.tensor(process_ids)
    #     return self.process_embeddings(process_ids)

#### Data Loader

In [160]:
dataset = TimeSeriesDataset(df_subset)
process_embedding = ProcessEmbedding(df_subset, embedding_dim=84)

Time series shape: torch.Size([28]), Dimensions: 1
Time series shape: torch.Size([28]), Dimensions: 1
Time series shape: torch.Size([28]), Dimensions: 1
Time series shape: torch.Size([28]), Dimensions: 1
Time series shape: torch.Size([28]), Dimensions: 1
Time series shape: torch.Size([28]), Dimensions: 1
Time series shape: torch.Size([28]), Dimensions: 1
Time series shape: torch.Size([28]), Dimensions: 1
Time series shape: torch.Size([28]), Dimensions: 1
Time series shape: torch.Size([28]), Dimensions: 1
Time series shape: torch.Size([42]), Dimensions: 1
Time series shape: torch.Size([42]), Dimensions: 1
Time series shape: torch.Size([42]), Dimensions: 1
Time series shape: torch.Size([42]), Dimensions: 1
Time series shape: torch.Size([42]), Dimensions: 1
Time series shape: torch.Size([42]), Dimensions: 1
Time series shape: torch.Size([42]), Dimensions: 1
Time series shape: torch.Size([42]), Dimensions: 1
Time series shape: torch.Size([42]), Dimensions: 1
Time series shape: torch.Size([

#### Processing The Embedding

In [161]:
batch_size = 4
data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True) 

for hero_ids, time_series, lengths in data_loader:
    process_embeddings = process_embedding(hero_ids)
    # Use process_embeddings and time_series for training or inference

tensor([88, 71, 66, 26])
tensor([ 64,  18,  22, 106])
tensor([ 96,  72,  25, 129])
tensor([ 90,  63, 113,  84])
tensor([44, 69, 89, 97])
tensor([ 5, 96, 14, 96])
tensor([77, 17, 87, 61])
tensor([ 96, 129,  84,  17])
tensor([ 14,  10, 102,  14])
tensor([10, 69, 66, 51])
tensor([47, 14, 98, 20])
tensor([110, 114, 113,  79])
tensor([43, 19, 68, 26])
tensor([ 6, 90, 18, 66])
tensor([21, 23, 41, 53])
tensor([23, 71, 52, 91])
tensor([87, 21, 66, 86])
tensor([68, 19, 57,  6])
tensor([25, 45, 40, 61])
tensor([45,  5, 39, 19])
tensor([  1, 102,  26, 136])
tensor([ 80,  97,  75, 100])
tensor([71, 20, 14, 78])
tensor([ 4, 26, 53, 25])
tensor([ 51, 101,  67,  61])
tensor([ 26,  69,  88, 129])
tensor([ 21,  63, 106,  73])
tensor([ 54, 136,  19,  70])
tensor([ 59, 106,  76,   1])
tensor([ 26,  85,  20, 100])
tensor([ 5, 96, 13, 86])
tensor([ 84, 129,   7,   7])
tensor([ 10,  43, 136,  98])
tensor([12, 26, 23, 23])
tensor([102, 104,   6,  71])
tensor([21, 20, 21, 45])
tensor([ 84,  98, 128,  80])
ten

## Building LSTM Model

In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, process_embedding):
        super(LSTMModel, self).__init__() # ensures the correcty PyTorch class is also initialized

        self.hidden_size = hidden_size #hyper param 
        self.num_layers = num_layers #hyper param

        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True) # Actual LSTM creation
        self.fc = nn.Linear(hidden_size, output_size) # Linear Model creation
        self.process_embedding = process_embedding # Process Embedding


    def forward(self, hero_ids, time_series):
        batch_size = time_series.size(0) # pulling dims from the tensor
        seq_length = time_series.size(1) # pulling dims from the tensor
        
        # Get process embeddings for hero_ids
        process_embeddings = self.process_embedding(hero_ids)
        
        # Reshape process embeddings to match the input shape of LSTM
        process_embeddings = process_embeddings.unsqueeze(1).repeat(1, seq_length, 1)
        
        # Concatenate process embeddings with time series data
        input_data = torch.cat((process_embeddings, time_series), dim=-1)
        
        # Pack the padded sequences
        packed_input = pack_padded_sequence(input_data, lengths, batch_first=True, enforce_sorted=False)


        # Initialize hidden state and cell state
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_size)
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_size)
        
        # 
        packed_output, _ = self.lstm(packed_input, (h0, c0))

        # Unpack the output
        output, _ = pad_packed_sequence(packed_output, batch_first=True)

        # Take the last output of the LSTM
        out = self.fc(out[:, -1, :])
        
        return out

In [ ]:
time_series.shape

torch.Size([2, 28])

In [ ]:
input_size = process_embedding.embedding_dim + time_series.shape[-1]  # Number of features (embedding_dim + time_series_dim)
hidden_size = 64
num_layers = 2
output_size = 1  # Assuming you want to predict a single value

model = LSTMModel(input_size, hidden_size, num_layers, output_size, process_embedding)

#### Training the Model

In [ ]:
# Assuming you have the following variables:
# - train_hero_ids: List of hero_ids for training
# - train_time_series: List of time series data for training
# - train_targets: List of target values for training

train_hero_ids = torch.tensor(train_hero_ids)
train_time_series = torch.tensor(train_time_series)
train_targets = torch.tensor(train_targets)

### Non-padded Training Loop

In [ ]:
# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
batch_size = 32

for epoch in range(num_epochs):
    for i in range(0, len(train_hero_ids), batch_size):
        batch_hero_ids = train_hero_ids[i:i+batch_size]
        batch_time_series = train_time_series[i:i+batch_size]
        batch_targets = train_targets[i:i+batch_size]
        
        optimizer.zero_grad()
        outputs = model(batch_hero_ids, batch_time_series)
        loss = criterion(outputs, batch_targets)
        loss.backward()
        optimizer.step()

### Padded Training Loop

In [ ]:

for epoch in range(num_epochs):
    for hero_ids, padded_time_series, lengths in data_loader:
        optimizer.zero_grad()

        # Sort the sequences by length in descending order
        lengths, sorted_indices = lengths.sort(descending=True)
        hero_ids = hero_ids[sorted_indices]
        padded_time_series = padded_time_series[sorted_indices]

        # Forward pass
        outputs = model(hero_ids, padded_time_series, lengths)

        # Compute loss and perform backpropagation
        loss = criterion(outputs, targets[sorted_indices])
        loss.backward()
        optimizer.step()

#### Evaluating Model

In [ ]:
# Assuming you have the following variables:
# - test_hero_ids: List of hero_ids for testing
# - test_time_series: List of time series data for testing
# - test_targets: List of target values for testing

model.eval()
with torch.no_grad():
    test_hero_ids = torch.tensor(test_hero_ids)
    test_time_series = torch.tensor(test_time_series)
    test_targets = torch.tensor(test_targets)
    
    outputs = model(test_hero_ids, test_time_series)
    loss = criterion(outputs, test_targets)
    
    # Calculate evaluation metrics (e.g., MSE, MAE)
    # ...

## Running LSTM

In [ ]:
### Parameters
#embedding_dim = 
test_size = .30
#shuffle = 
num_epochs = 20


### Train Test Split

In [ ]:
train_df, test_df = train_test_split(df_subset, test_size=test_size, shuffle=False)

#### Dataset and Data Load

In [ ]:
train_dataset = TimeSeriesDataset(train_df)
test_dataset = TimeSeriesDataset(test_df)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
model.train()
for epoch in range(num_epochs):
    for hero_ids, time_series in train_loader:
        optimizer.zero_grad()
        process_embeddings = process_embedding(hero_ids)
        output = model(process_embeddings, time_series)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

NameError: name 'model' is not defined

## Keras Tensorflow Version

In [ ]:
df_subset = df_match[['hero_id', 'gold_t']].copy()

df_subset = df_subset.explode('gold_t').reset_index(drop=True)

#df_pivoted = df_subset.pivot_table(index=df_subset.index, columns='hero_id', values='gold_t')

df['gold_t'].apply(lambda x: x.split(','))
#df_subset = df_subset.T
df_subset
#df_pivoted

,hero_id,gold_t
0,26,0
1,26,170
2,26,260
3,26,350
4,26,440
...,...,...
346645,19,10287
346646,19,11317
346647,19,12596
346648,19,13275


Splitting a sequence in the first step for prepping data for an LSTM model.
This is following along with : https://machinelearningmastery.com/how-to-develop-lstm-models-for-time-series-forecasting/


Below we create a function that will create sample sequences based on the number of steps we feed the model before expecting an output. For example, below uses 3 steps fed to predict 1 step out.

In [ ]:
# univariate data preparation
from numpy import array

# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

# define input sequence
raw_seq = gold_t
# choose a number of time steps
n_steps = 3
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])

[  0 170 260] 350
[170 260 350] 440
[260 350 440] 827
[350 440 827] 1051
[ 440  827 1051] 1141
[ 827 1051 1141] 1278
[1051 1141 1278] 1368
[1141 1278 1368] 1593
[1278 1368 1593] 2076
[1368 1593 2076] 2166
[1593 2076 2166] 2261
[2076 2166 2261] 2609
[2166 2261 2609] 2704
[2261 2609 2704] 2834
[2609 2704 2834] 3702
[2704 2834 3702] 4056
[2834 3702 4056] 4214
[3702 4056 4214] 4309
[4056 4214 4309] 4640
[4214 4309 4640] 4806
[4309 4640 4806] 5091
[4640 4806 5091] 5451
[4806 5091 5451] 5635
[5091 5451 5635] 5790
[5451 5635 5790] 5943


The LSTM model expects data in the form of [samples,timesteps,features]. Our current X shape is in the form of [samples, timesteps]. i.e. the first row in X is the first sample being [0 . 170, 260] and its timesteps are 3, one for each column.

 Recall that we are using tensors here (3 Dimensional arrays) and will use .Reshape() methods to conform shape to LSTM.


In this case we have one time series. This equates to 1 "feature" to the LSTM.

For this one feature, we've chopped it into 25 samples of 3 time-steps (what we chose in the split_sequence function).

Thus we re-shape our X from [25, 3] into a [25, 3, 1].


In [ ]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))

X

array([[[   0],
        [ 170],
        [ 260]],

       [[ 170],
        [ 260],
        [ 350]],

       [[ 260],
        [ 350],
        [ 440]],

       [[ 350],
        [ 440],
        [ 827]],

       [[ 440],
        [ 827],
        [1051]],

       [[ 827],
        [1051],
        [1141]],

       [[1051],
        [1141],
        [1278]],

       [[1141],
        [1278],
        [1368]],

       [[1278],
        [1368],
        [1593]],

       [[1368],
        [1593],
        [2076]],

       [[1593],
        [2076],
        [2166]],

       [[2076],
        [2166],
        [2261]],

       [[2166],
        [2261],
        [2609]],

       [[2261],
        [2609],
        [2704]],

       [[2609],
        [2704],
        [2834]],

       [[2704],
        [2834],
        [3702]],

       [[2834],
        [3702],
        [4056]],

       [[3702],
        [4056],
        [4214]],

       [[4056],
        [4214],
        [4309]],

       [[4214],
        [4309],
        [4640]],



In [ ]:
# univariate lstm example
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

In [ ]:
# expected input data shape: (batch_size, timesteps, data_dim)
model = Sequential()
model.add(LSTM(32,input_shape=(3, 1))) 

AttributeError: module 'keras.src.backend' has no attribute 'Variable'

In [ ]:
# define model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps,n_features))) # 3 steps, 1 feature
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

AttributeError: module 'keras.src.backend' has no attribute 'Variable'